This is not used.....

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import (Convolution2D, MaxPooling2D, Convolution3D,
                          MaxPooling3D)
from keras import backend as K

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
import os.path as op
from glob import glob

In [4]:
import dipy.data as dpd
import nibabel as nib

In [5]:
dpd.fetch_bundles_2_subjects()

Creating new folder /Users/jpowen/.dipy/exp_bundles_and_maps
Data size is approximately 234MB
Download Progress: [##################################################################################################] 100.00%  of 233.75 MBFiles successfully downloaded to /Users/jpowen/.dipy/exp_bundles_and_maps


({'bundles_2_subjects.tar.gz': ('https://digital.lib.washington.edu/researchworks/bitstream/handle/1773/38477/bundles_2_subjects.tar.gz',
   '97756fbef11ce2df31f1bedf1fc7aac7')},
 '/Users/jpowen/.dipy/exp_bundles_and_maps')

In [6]:
DATA_SUB1 = '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1'

In [8]:
bundle_fnames = glob(op.join(DATA_SUB1, 'bundles/*.trk'))
bundle_fnames

['/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_6.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cst.left.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_7.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_5.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_slf2.right.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.left.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.right.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_ifof.left.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_4.trk',
 '/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_1.trk',
 '/Users/jpowe

In [9]:
t1_warped_img = nib.load(op.join(DATA_SUB1, 't1_warped.nii.gz'))

In [12]:
sl_sum = 0
for b_idx, bundle in enumerate(bundle_fnames):
    tgram = nib.streamlines.load(op.join(DATA_SUB1, 'bundles', bundle))
    print(bundle, len(tgram.streamlines))
    sl_sum += len(tgram.streamlines)


0
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_6.trk 3620
1
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cst.left.trk 2101
2
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_7.trk 703
3
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_5.trk 520
4
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_slf2.right.trk 1441
5
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.left.trk 343
6
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_mdlf.right.trk 161
7
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_ifof.left.trk 543
8
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_4.trk 1970
9
/Users/jpowen/.dipy/exp_bundles_and_maps/bundles_2_subjects/subj_1/bundles/bundles_cc_

In [10]:
bundle_arr = np.memmap('/home/ariel/data/bundles/npy/bundles.npy', mode='r', 
                       shape=((sl_sum, ) + t1_warped_img.shape + (1,)), dtype=bool)
one_hot_arr = np.memmap('/home/ariel/data/bundles/npy/one_hot.npy', mode='r', 
                        shape=(sl_sum, len(bundle_fnames)), dtype=bool)

In [11]:
## For now, let's look at a tiny subset: 
bundle_arr = bundle_arr[::100]
one_hot_arr = one_hot_arr[::100]

In [12]:
bundle_arr.shape

(462, 256, 256, 150, 1)

In [13]:
batch_size = 1
num_classes = 27
epochs = 12

In [14]:
# input image dimensions
img_x, img_y, img_z = t1_warped_img.shape
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [15]:
y_test.shape, y_train.shape

((10000,), (60000,))

In [16]:
10000 / 70000

0.14285714285714285

In [17]:
import sklearn.model_selection as ms 

In [18]:
sp = ms.ShuffleSplit(n_splits=1, test_size=0.15)

In [19]:
foo = sp.split(bundle_arr)

In [20]:
for train_index, test_index in sp.split(bundle_arr):
    tr = train_index
    te = test_index

In [21]:
tr.shape, te.shape

((392,), (70,))

In [22]:
import tempfile 

In [23]:
tmpdir = tempfile.TemporaryDirectory()

In [24]:
bundle_train_file = op.join(tmpdir.name, 'bundles_train.npy')
bundle_test_file = op.join(tmpdir.name, 'bundles_test.npy')
one_hot_train_file = op.join(tmpdir.name, 'one_hot_train.npy')
one_hot_test_file = op.join(tmpdir.name, 'one_hot_test.npy')

In [25]:
# bundle_train_arr = np.memmap(bundle_train_file, mode='w+', 
#                              shape=(tr.shape + t1_warped_img.shape + (1,)), dtype=bool)

In [26]:
# bundle_test_arr = np.memmap(bundle_test_file, mode='w+', 
#                             shape=(te.shape + t1_warped_img.shape + (1,)), dtype=bool)

In [27]:
# one_hot_train_arr = np.memmap(one_hot_train_file, mode='w+', 
#                               shape=(tr.shape[0], len(bundle_fnames)), dtype=bool)

In [28]:
# one_hot_test_arr = np.memmap(one_hot_test_file, mode='w+', 
#                              shape=(te.shape[0], len(bundle_fnames)), dtype=bool)

In [29]:
bundle_train_arr = np.zeros((tr.shape + t1_warped_img.shape + (1,)), dtype=bool)
bundle_test_arr = np.zeros((te.shape + t1_warped_img.shape + (1,)), dtype=bool)
one_hot_train_arr = np.zeros((tr.shape[0], len(bundle_fnames)), dtype=bool)
one_hot_test_arr = np.zeros((te.shape[0], len(bundle_fnames)), dtype=bool)

In [30]:
for ii, idx in enumerate(tr):
    bundle_train_arr[ii] = bundle_arr[idx]
    one_hot_train_arr[ii] = one_hot_arr[idx]

In [31]:
for ii, idx in enumerate(te):
    bundle_test_arr[ii] = bundle_arr[idx]
    one_hot_test_arr[ii] = one_hot_arr[idx]

In [32]:
input_shape = (img_x, img_y, img_z, 1)

In [33]:
model = Sequential()
model.add(Convolution3D(32, kernel_size=(3, 3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Convolution3D(64, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [34]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [35]:
bundle_train_arr.shape, one_hot_train_arr.shape

((392, 256, 256, 150, 1), (392, 27))

In [36]:
def generate_samples():
    while 1:
        for ii in range(bundle_train_arr.shape[0]):
            yield bundle_train_arr[ii:ii+1], one_hot_train_arr[ii:ii+1]

In [37]:
def generate_validation():
    while 1:
        for ii in range(bundle_test_arr.shape[0]):
            yield bundle_test_arr[ii:ii+1], one_hot_test_arr[ii:ii+1]

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def __init__(self, directory, descript, augment):
        self.director = directory
        self.descript = descript
        self.augment = augment

    def on_train_begin(self, logs={}):
        self.losses = []
        self.valid = []
        self.lastiter = 0

    def on_epoch_end(self, batch, logs={}):
        self.lastiter += 1
        with open("%s/history.txt" % self.d, "a") as fout:
                for metric in ["loss", "acc" ]:
                        fout.write("%s\t%d\ttrain\t%d\t%s\t%.6f\n" % (self.descript, self.augment, self.lastiter, metric, logs.get(metric)))
                for metric in ["val_loss", "val_acc"]:
                        fout.write("%s\t%d\tvalid\t%d\t%s\t%.6f\n" % (self.descript, self.augment, self.lastiter, metric, logs.get(metric)))

    def on_batch_end(self, batch, logs={}):
        self.lastiter += 1
        with open("%s/history.txt" % self.d, "a") as fout:
                for metric in ["loss", "acc"]:
                        fout.write("%s\t%d\ttrain\t%d\t%s\t%.6f\n" % (self.descript, self.augment, self.lastiter, metric, logs.get(metric)))


In [ ]:
model.fit_generator(generate_samples(), 
                    steps_per_epoch=1,
                    epochs=1,
                    verbose=1,
                    validation_data=generate_validation(), 
                    validation_steps=1)

Epoch 1/1


In [ ]:
# model.fit(bundle_train_arr, 
#           one_hot_train_arr,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(bundle_test_arr, one_hot_test_arr))

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.fit_generator?